In [24]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import eli5
from eli5.sklearn import PermutationImportance

In [2]:
feature_set = pd.read_csv('/feature_matrix_w_position.csv')
feature_set = feature_set.drop(columns = ['Unnamed: 0'], axis = 1)

feature_set = feature_set.replace('H', 1)
feature_set = feature_set.replace('A', 0)

feature_set = feature_set.dropna()


In [3]:
feature_set_test_date = feature_set[(feature_set['season']==2019) & (feature_set['game_id'] >= 196) & (feature_set['game_id'] <= 203)]

feature_set_pre_test_date = feature_set[(feature_set['season']<2019) | (feature_set['season'] == 2019) & (feature_set['game_id'] < 196)]

In [5]:
X_pre = feature_set_pre_test_date.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season','primaryPositioncode','primaryPositionname','primaryPositiontype'], axis = 1)
y_pre = feature_set_pre_test_date['dfs_points'].tolist()

feature_set_forward = feature_set_pre_test_date.loc[feature_set_pre_test_date['primaryPositioncode'].isin(['L','R','C'])]
X_forward = feature_set_forward.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season','primaryPositioncode','primaryPositionname','primaryPositiontype'], axis = 1)
y_forward = feature_set_forward['dfs_points'].tolist()
                                      
feature_set_defense = feature_set_pre_test_date.loc[feature_set_pre_test_date['primaryPositioncode'].isin(['D'])]
X_defense = feature_set_defense.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season','primaryPositioncode','primaryPositionname','primaryPositiontype'], axis = 1)
y_defense = feature_set_defense['dfs_points'].tolist()

X_train_forward, X_test_forward, y_train_forward, y_test_forward = train_test_split(X_forward, y_forward, test_size=0.2, random_state=42)
X_train_defense, X_test_defense, y_train_defense, y_test_defense = train_test_split(X_defense, y_defense, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_pre, y_pre, test_size=0.2, random_state=42)


### Neural Net with 3 HL and Standard scaler

In [6]:
# import os 
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [7]:
import tensorflow as tf 
gpus= tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(gpus[0], True) 

In [8]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import tensorflow as tf

Using TensorFlow backend.


In [9]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(15, input_dim=71, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=40, batch_size=64, verbose=1)
kfold = KFold(n_splits=5)
results = cross_val_score(estimator, X_train_forward, y_train_forward, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/40
52384/52384 [==============================] - 2s 29us/step - loss: 55.0037
Epoch 2/40
52384/52384 [==============================] - 1s 23us/step - loss: 48.0921
Epoch 3/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.9531
Epoch 4/40
52384/52384 [==============================] - 1s 23us/step - loss: 47.8765
Epoch 5/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.7310
Epoch 6/40
52384/52384 [==============================] - 1s 23us/step - loss: 47.8483
Epoch 7/40
52384/52384 [==============================] - 1s 23us/step - loss: 47.6108
Epoch 8/40
52384/52384 [==============================] - 1s 23us/step - loss: 47.5784
Epoch 9/40
52384/52384 [==============================] - 1s 23us/step - loss: 47.5201
Epoch 10/40
52384/52384 [==============================] - 1s 23us/step - loss: 47.5788
Epoch 11/40
52384/52384 [==============================] - 1s 23us/step - loss: 47.4469
Epoch 12/40
52384/52384 [================

52384/52384 [==============================] - 1s 23us/step - loss: 47.1416
Epoch 14/40
52384/52384 [==============================] - 1s 23us/step - loss: 47.1013
Epoch 15/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.0812
Epoch 16/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.0549
Epoch 17/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.0289
Epoch 18/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.0133
Epoch 19/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.0038
Epoch 20/40
52384/52384 [==============================] - 1s 22us/step - loss: 46.9491
Epoch 21/40
52384/52384 [==============================] - 1s 23us/step - loss: 46.9220
Epoch 22/40
52384/52384 [==============================] - 1s 23us/step - loss: 46.8949
Epoch 23/40
52384/52384 [==============================] - 1s 23us/step - loss: 46.9182
Epoch 24/40
52384/52384 [===================

52384/52384 [==============================] - 1s 22us/step - loss: 47.2413
Epoch 26/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.2917
Epoch 27/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.2169
Epoch 28/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.2451
Epoch 29/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.2303
Epoch 30/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.2678
Epoch 31/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.1959
Epoch 32/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.2469
Epoch 33/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.2176
Epoch 34/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.1843
Epoch 35/40
52384/52384 [==============================] - 1s 22us/step - loss: 47.2227
Epoch 36/40
52384/52384 [===================

### Offense

In [10]:
forward_model = KerasRegressor(build_fn=baseline_model, epochs=40, batch_size=64, verbose=1)
forward_model.fit(X_train_forward, y_train_forward)
y_pred_train = forward_model.predict(X_train_forward)
y_pred_test = forward_model.predict(X_test_forward)
mse_test = mean_squared_error(y_test_forward, y_pred_test)
mse_train = mean_squared_error(y_train_forward, y_pred_train)
print('Test: ' + str(mse_test))
print('Train: ' + str(mse_train))

Epoch 1/40
65480/65480 [==============================] - 1s 23us/step - loss: 48.1201
Epoch 2/40
65480/65480 [==============================] - 1s 22us/step - loss: 47.8006
Epoch 3/40
65480/65480 [==============================] - 1s 22us/step - loss: 47.7268
Epoch 4/40
65480/65480 [==============================] - 1s 22us/step - loss: 47.6072
Epoch 5/40
65480/65480 [==============================] - 1s 22us/step - loss: 47.5604
Epoch 6/40
65480/65480 [==============================] - 1s 22us/step - loss: 47.5182
Epoch 7/40
65480/65480 [==============================] - 1s 22us/step - loss: 47.4293
Epoch 8/40
65480/65480 [==============================] - 2s 23us/step - loss: 47.3455
Epoch 9/40
65480/65480 [==============================] - 1s 22us/step - loss: 47.2315
Epoch 10/40
65480/65480 [==============================] - 1s 22us/step - loss: 47.2686
Epoch 11/40
65480/65480 [==============================] - 1s 23us/step - loss: 47.2401
Epoch 12/40
65480/65480 [================

### Defense 

In [11]:
defense_model = KerasRegressor(build_fn=baseline_model, epochs=40, batch_size=64, verbose=1)
defense_model.fit(X_train_defense, y_train_defense)
y_pred_train = defense_model.predict(X_train_defense)
y_pred_test = defense_model.predict(X_test_defense)
mse_test = mean_squared_error(y_test_defense, y_pred_test)
mse_train = mean_squared_error(y_train_defense, y_pred_train)
print('Test: ' + str(mse_test))
print('Train: ' + str(mse_train))

Epoch 1/40
35273/35273 [==============================] - 1s 24us/step - loss: 35.1819
Epoch 2/40
35273/35273 [==============================] - 1s 22us/step - loss: 30.7570
Epoch 3/40
35273/35273 [==============================] - 1s 23us/step - loss: 30.5979
Epoch 4/40
35273/35273 [==============================] - 1s 23us/step - loss: 30.4844
Epoch 5/40
35273/35273 [==============================] - 1s 23us/step - loss: 30.4701
Epoch 6/40
35273/35273 [==============================] - 1s 24us/step - loss: 30.2623
Epoch 7/40
35273/35273 [==============================] - 1s 24us/step - loss: 30.3507
Epoch 8/40
35273/35273 [==============================] - 1s 23us/step - loss: 30.2257
Epoch 9/40
35273/35273 [==============================] - 1s 23us/step - loss: 30.3363
Epoch 10/40
35273/35273 [==============================] - 1s 22us/step - loss: 30.1999
Epoch 11/40
35273/35273 [==============================] - 1s 22us/step - loss: 30.2093
Epoch 12/40
35273/35273 [================

### Entire Dataset

In [12]:
full_dataset_model = KerasRegressor(build_fn=baseline_model, epochs=40, batch_size=64, verbose=1)
full_dataset_model.fit(X_train, y_train)
y_pred_train = full_dataset_model.predict(X_train)
y_pred_test = full_dataset_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
mse_train = mean_squared_error(y_train, y_pred_train)
print('Test: ' + str(mse_test))
print('Train: ' + str(mse_train))

Epoch 1/40
100754/100754 [==============================] - 2s 24us/step - loss: 42.8059
Epoch 2/40
100754/100754 [==============================] - 2s 24us/step - loss: 42.0279
Epoch 3/40
100754/100754 [==============================] - 2s 23us/step - loss: 41.7345
Epoch 4/40
100754/100754 [==============================] - 2s 23us/step - loss: 41.6286
Epoch 5/40
100754/100754 [==============================] - 2s 23us/step - loss: 41.4496
Epoch 6/40
100754/100754 [==============================] - 2s 23us/step - loss: 41.3891
Epoch 7/40
100754/100754 [==============================] - 2s 23us/step - loss: 41.3647
Epoch 8/40
100754/100754 [==============================] - 2s 22us/step - loss: 41.3521
Epoch 9/40
100754/100754 [==============================] - 2s 23us/step - loss: 41.3617
Epoch 10/40
100754/100754 [==============================] - 2s 23us/step - loss: 41.2416
Epoch 11/40
100754/100754 [==============================] - 2s 24us/step - loss: 41.2852
Epoch 12/40
100754/

In [25]:
perm = PermutationImportance(full_dataset_model, random_state=1).fit(X_train,y_train)
eli5.show_weights(perm, feature_names = X_train.columns.tolist())

100754/100754 [==============================] - 1s 12us/step


100754/100754 [==============================] - 1s 14us/step


100754/100754 [==============================] - 1s 10us/step


Weight,Feature
0.5776 ± 0.0341,pp_toi_season
0.3941 ± 0.0261,shots_20
0.3347 ± 0.0164,shots_season
0.3076 ± 0.0216,pp_toi_20
0.2443 ± 0.0092,shots_10
0.1527 ± 0.0155,es_toi_1
0.1452 ± 0.0082,es_toi_5
0.1370 ± 0.0069,shots_5
0.0989 ± 0.0103,homeOrAway
0.0530 ± 0.0098,pp_toi_5


### Goalies

In [13]:
feature_set_G = pd.read_csv('goalie_feature_matrix.csv')
feature_set_G = feature_set_G.drop(columns = ['Unnamed: 0'], axis = 1)

feature_set_G = feature_set_G.replace('H', 1)
feature_set_G = feature_set_G.replace('A', 0)

feature_set_G = feature_set_G.dropna()

feature_set_G.head(10)

,player_id,game_id,season,homeOrAway,saves_1,saves_5,saves_10,saves_20,saves_season,goals_against_1,...,sh_shots_5,sh_shots_10,sh_shots_20,sh_shots_season,sh_save_percent_1,sh_save_percent_5,sh_save_percent_10,sh_save_percent_20,sh_save_percent_season,dfs_points
24,8466141,454,2015,0,22.0,23.2,25.2,26.90,27.708333,2.0,...,0.2,0.5,0.45,0.625000,100.0,100.000000,87.5,93.750000,95.000000,25.6
25,8466141,479,2015,0,33.0,24.0,24.7,26.95,27.920000,1.0,...,0.6,0.5,0.50,0.680000,100.0,100.000000,100.0,93.750000,95.454545,26.8
30,8466141,572,2015,1,29.0,22.6,23.3,25.10,27.033333,1.0,...,0.4,0.5,0.45,0.633333,100.0,100.000000,100.0,91.666667,95.833333,41.0
31,8466141,589,2015,0,40.0,25.8,24.1,25.85,27.451613,0.0,...,0.6,0.6,0.50,0.645161,100.0,100.000000,100.0,92.857143,96.153846,24.2
33,8466141,632,2015,0,30.0,29.0,26.9,26.90,27.636364,2.0,...,1.0,0.8,0.60,0.666667,100.0,100.000000,100.0,93.750000,96.428571,10.4
36,8466141,707,2015,1,29.0,25.6,25.7,25.10,27.194444,2.0,...,0.6,0.6,0.50,0.638889,100.0,100.000000,100.0,100.000000,96.666667,28.9
38,8466141,736,2015,1,38.0,26.4,27.7,26.15,27.473684,2.0,...,0.4,0.7,0.50,0.631579,100.0,100.000000,100.0,100.000000,96.875000,17.9
41,8466141,795,2015,0,32.0,31.0,28.3,26.20,27.658537,3.0,...,0.4,0.5,0.55,0.609756,100.0,100.000000,100.0,100.000000,97.058824,13.0
44,8466141,877,2015,1,23.0,26.2,25.8,26.60,27.204545,5.0,...,0.6,0.5,0.60,0.613636,50.0,75.000000,87.5,93.750000,94.444444,22.8
45,8466141,909,2015,1,29.0,24.8,27.6,26.40,27.244444,1.0,...,0.8,0.6,0.55,0.622222,100.0,83.333333,90.0,93.750000,94.736842,14.4


In [14]:
feature_set_test_date_G = feature_set_G[(feature_set_G['season']==2019) & (feature_set_G['game_id'] >= 196) & (feature_set_G['game_id'] <= 203)]

feature_set_pre_test_date_G = feature_set_G[(feature_set_G['season']<2019) | (feature_set_G['season'] == 2019) & (feature_set_G['game_id'] < 196)]


In [15]:
X_G = feature_set_pre_test_date_G.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season'], axis = 1)
y_G = feature_set_pre_test_date_G['dfs_points'].tolist()

X_train_G, X_test_G, y_train_G, y_test_G = train_test_split(X_G, y_G, test_size=0.2, random_state=42)

In [16]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(15, input_dim=66, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
goalie_model = KerasRegressor(build_fn=baseline_model, epochs=40, batch_size=64, verbose=1)
goalie_model.fit(X_train_G, y_train_G)
y_pred_train = goalie_model.predict(X_train_G)
y_pred_test = goalie_model.predict(X_test_G)
mse_test = mean_squared_error(y_test_G, y_pred_test)
mse_train = mean_squared_error(y_train_G, y_pred_train)
print('Test: ' + str(mse_test))
print('Train: ' + str(mse_train))

Epoch 1/40
3089/3089 [==============================] - 0s 36us/step - loss: 145.2677
Epoch 2/40
3089/3089 [==============================] - 0s 23us/step - loss: 114.5834
Epoch 3/40
3089/3089 [==============================] - 0s 22us/step - loss: 114.7871
Epoch 4/40
3089/3089 [==============================] - 0s 22us/step - loss: 114.7272
Epoch 5/40
3089/3089 [==============================] - 0s 22us/step - loss: 114.6175
Epoch 6/40
3089/3089 [==============================] - 0s 21us/step - loss: 114.5962
Epoch 7/40
3089/3089 [==============================] - 0s 22us/step - loss: 115.1068
Epoch 8/40
3089/3089 [==============================] - 0s 22us/step - loss: 114.5844
Epoch 9/40
3089/3089 [==============================] - 0s 22us/step - loss: 114.8107
Epoch 10/40
3089/3089 [==============================] - 0s 22us/step - loss: 114.5596
Epoch 11/40
3089/3089 [==============================] - 0s 22us/step - loss: 114.7042
Epoch 12/40
3089/3089 [=============================

In [17]:
feature_set_forward = feature_set_test_date.loc[feature_set_test_date['primaryPositioncode'].isin(['L','R','C'])]
X_forward = feature_set_forward.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season','primaryPositioncode','primaryPositionname','primaryPositiontype'], axis = 1)
y_forward = feature_set_forward['dfs_points'].tolist()
                                      
feature_set_defense = feature_set_test_date.loc[feature_set_test_date['primaryPositioncode'].isin(['D'])]
X_defense = feature_set_defense.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season','primaryPositioncode','primaryPositionname','primaryPositiontype'], axis = 1)
y_defense = feature_set_defense['dfs_points'].tolist()

X_G = feature_set_test_date_G.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season'], axis = 1)
y_G = feature_set_test_date_G['dfs_points'].tolist()

In [18]:
forward_prediction = (forward_model.predict(X_forward))
defense_prediction = (defense_model.predict(X_defense))
goalie_prediction = (goalie_model.predict(X_G))
forward_final = pd.DataFrame(feature_set_forward)
forward_final['NN_DFS_Prediction'] = forward_prediction
defense_final = pd.DataFrame(feature_set_defense)
defense_final['NN_DFS_Prediction'] = defense_prediction
goalie_final = pd.DataFrame(feature_set_test_date_G)
goalie_final['NN_DFS_Prediction'] = goalie_prediction
final_predictions = pd.concat([forward_final,defense_final,goalie_final])

7/7 [==============================] - 0s 0us/step


C:\Users\Vijay Gentyala\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [19]:
final_predictions=final_predictions[['player_id','NN_DFS_Prediction']]

In [20]:
final_predictions

,player_id,NN_DFS_Prediction
1732,8466138,6.240890
2124,8466139,6.974981
14045,8470047,6.519567
15032,8470144,4.573442
16665,8470257,5.838479
...,...,...
5647,8473541,14.128203
6492,8474593,14.653834
7099,8474651,14.222071
10218,8476412,14.811571


In [21]:
final_predictions.to_csv('final_predictions.csv')